# 2 Customer Onboarding with predefined sequential conversation

## Setup

In [1]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
llm_config = {"model": "gpt-4o", "api_key":OPENAI_API_KEY, "temperature":1.0,"cache_seed": None}

from autogen import ConversableAgent

## Use case
Let's dive into a possible real scenario. We are going to set up a system in which virtual agents help new patients register and receive initial health advice based on the information they collect from the user.

### Agents creation
Let's create our agents, specializing them in specific tasks.

The first agent is the 'personal information' agent, responsible for collecting basic user information.

In [2]:
# Agent specialized in gathering patient personal information to onboard them into the health service
personal_information_agent = ConversableAgent(
    name="Personal_Information_Agent",
    system_message='''
    You are an agent specializing in onboarding new patients.
    Your role is to guide them in getting started with our health service.
    Your only responsibility is to collect the patient's name and place of residence. 
    Do not request any additional data. 
    ''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER", # <-- agent acts autonomously
)

The second agent is the 'symptoms-registering' agent, responsible for recording the patient's initial symptoms of the illness.

In [3]:
# Agent specialized in gathering patient's initial symptoms
health_information_agent = ConversableAgent(
    name="Health_Information_Agent",
    system_message='''
    You are an agent specializing in onboarding new patients.
    Your role is to guide them in getting started with our health service.
    Your only responsibility is to gather information about the patient's current symptoms.
    Do not request additional data.
    Once you have completed the collection, respond with 'INFORMATION COLLECTED'
    ''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER", # <-- agent acts autonomously
)

The third agent is the 'health advice' agent, responsible for providing initial recommendations based on the patient's symptoms

In [ ]:
# Agent specialized in providing initial advice based on patient's personal information.
health_advice_agent = ConversableAgent(
    name="Health_Advice_Agent",
    system_message='''
    You are a health care agent dedicated to providing initial advice based on patient's personal information 
    and the their early symptoms of illness.
    You offer general suggestions and always remember to consult a physician. 
    Your goal is to be formal, clear and reassuring.
    Once you have provided your initial assistance, respond with 'CONSULTATION TERMINATED'.
    ''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER", # <-- agent acts autonomously
    is_termination_msg=lambda msg: "consultation terminated" in msg.get("content").lower(),
)

Finally we define a special agent that acts on behalf of the patient

In [5]:
patient_proxy_agent = ConversableAgent(
    name="patient_proxy_agent",
    llm_config=False, # NO LLM
    code_execution_config=False,
    human_input_mode="ALWAYS", # <-- agent always ask for human intervention
    is_termination_msg=lambda msg: "information collected" in msg.get("content").lower() or "consultation terminated" in msg.get("content").lower(),
)

## Creating tasks

Now, we can set a series of chats among the agents to facilitate the onboarding process.

In [6]:
chats = [
    {
        "sender": personal_information_agent,
        "recipient": patient_proxy_agent,
        "message": "Hello, I'm here to help you with initial care with our health service. Could you please tell me your name and your residential city?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": "Return the patient's personal information in a single JSON object: {'name': '', 'city': ''}", # <-- different way to summarize content
        },
        "max_turns": 2,
        "clear_history": True
    },
    {
        "sender": health_information_agent,
        "recipient": patient_proxy_agent,
        "message": "Great! Could you tell me your symptoms?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history": True
    },
    {
        "sender": patient_proxy_agent,
        "recipient": health_advice_agent,
        "message": "What initial treatments can help me recover as soon as possible?",
        "max_turns": 1,
        "clear_history":True,
        "summary_method": "reflection_with_llm",
    },
]

## Start the onboarding process

In [7]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Personal_Information_Agent (to patient_proxy_agent):

Hello, I'm here to help you with initial care with our health service. Could you please tell me your name and your residential city?

--------------------------------------------------------------------------------


c:\Users\ASUS\Anaconda3\envs\pydata_autogen\Lib\site-packages\autogen\agentchat\chat.py:57: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


patient_proxy_agent (to Personal_Information_Agent):

daniele

--------------------------------------------------------------------------------
Personal_Information_Agent (to patient_proxy_agent):

Thank you, Daniele! Could you also let me know the city you reside in?

--------------------------------------------------------------------------------
patient_proxy_agent (to Personal_Information_Agent):

i reside in rome

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Health_Information_Agent (to patient_proxy_agent):

Great! Could you tell me your symptoms?
Context: 
{'name': 'Daniele', 'city': 'Rome'}

--------------------------------------------------------------------------------
patient_proxy_agent (to Health_Information_Agent):

my knee is hurting

----------------

## Print out the summary

In [8]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

{'name': 'Daniele', 'city': 'Rome'}


Daniele from Rome has a knee that is hurting.


Daniele from Rome is advised to rest the knee, apply ice, use a compression bandage, elevate the knee, take over-the-counter pain relief, and perform gentle stretching and exercises. It is recommended to consult a physician for tailored advice and further assessment if needed.




## Print out the cost

In [9]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 0.000885, 'gpt-4o-2024-08-06': {'cost': 0.000885, 'prompt_tokens': 222, 'completion_tokens': 33, 'total_tokens': 255}}, 'usage_excluding_cached_inference': {'total_cost': 0.000885, 'gpt-4o-2024-08-06': {'cost': 0.000885, 'prompt_tokens': 222, 'completion_tokens': 33, 'total_tokens': 255}}}


{'usage_including_cached_inference': {'total_cost': 0.000295, 'gpt-4o-2024-08-06': {'cost': 0.000295, 'prompt_tokens': 70, 'completion_tokens': 12, 'total_tokens': 82}}, 'usage_excluding_cached_inference': {'total_cost': 0.000295, 'gpt-4o-2024-08-06': {'cost': 0.000295, 'prompt_tokens': 70, 'completion_tokens': 12, 'total_tokens': 82}}}


{'usage_including_cached_inference': {'total_cost': 0.0042525, 'gpt-4o-2024-08-06': {'cost': 0.0042525, 'prompt_tokens': 461, 'completion_tokens': 310, 'total_tokens': 771}}, 'usage_excluding_cached_inference': {'total_cost': 0.0042525, 'gpt-4o-2024-08-06': {'cost': 0.0042525, 'prompt_tokens': 461, 'completion_to